## 大语言模型基础

本章将首先介绍 Transformer 结构，并在此基础上介绍生成式预训练语言模型 GPT、大语言模型网络结构和注意力机制优化以及相关实践。

### Transformer 模型

Transformer结构及代码这里不再赘述，放一张结构图即可  

<img src="./images/transformer.png" style="zoom:60%;" /> 

### 生成式语言模型GPT

GPT是由OPENAI提出的一种生成式预训练语言模型，由多层Transformer组成(只包含Decoder)的单向语言模型，其结构为:  

<img src="./images/gpt.png" style="zoom:70%;" />  

GPT的模型的训练过程主要包括: 无监督预训练和有监督下游任务微调  

#### 无监督预训练

GPT 采用生成式预训练方法，单向意味着模型只能从左到右或从右到左对文本序列建模，所采用的`Transformer`结构和解码策略保证了输入文本每个位置只能依赖过去时刻的信息。

#### 有监督下游任务微调

通过无监督语言模型预训练，使得GPT模型具备了一定的通用语义表示能力。下游任务微调（Downstream Task Fine-tuning）的目的是在通用语义表示基础上，根据下游任务的特性进行适配。下游任务通常需要利用有标注数据集进行训练，每个样例由输入长度为n的文本序列和对应的标签y构成。

下游任务在微调过程中，针对任务目标进行优化，很容易使得模型遗忘预训练阶段所学习到的通用语义知识表示，从而损失模型的通用性和泛化能力，造成灾难性遗忘（Catastrophic Forgetting）问题。因此，通常会采用混合预训练任务损失和下游微调损失的方法来缓解上述问题。

#### 预训练语言模型实践

通过HuggingFace提供的数据集，以Bert模型为例，构建预训练模型

> 准备数据集

书中是直接通过代码下载HuggingFace的数据集，但是实际操作中数据集很大而且下载很慢，尽管配置了[hf-mirror](https://hf-mirror.com/)的镜像源速度也不快。所以最后选择从官网直接下载数据集，然后本地直接load就ok了

In [ ]:
pip install datasets

In [6]:
from datasets import load_dataset, concatenate_datasets

# 直接通过代码下载
#bookcorpus = load_dataset("bookcorpus", split="train", trust_remote_code=True)
#wiki = load_dataset("wikimedia/wikipedia", "20231101.en", split="train", trust_remote_code=True)

# 先从官网下载，然后本地load wikipedia数据集比较大只下载了部分数据
bookcorpus = load_dataset(path="data/bookcorpus", split="train")
bookcorpus = bookcorpus.select(range(2000000))  
print(bookcorpus)
wiki = load_dataset(path='data/wikimedia/wikipedia', split="train")
print(wiki)

HF google storage unreachable. Downloading and preparing it from source
Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 227.95it/s]


Dataset arrow downloaded and prepared to /Users/jiatianyu/.cache/huggingface/datasets/arrow/bookcorpus-f73b20ad1e10e33c/0.0.0/74f69db2c14c2860059d39860b1f400a03d11bf7fb5a8258ca38c501c878c137. Subsequent calls will reuse this data.
Dataset({
    features: ['text'],
    num_rows: 2000000
})


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 91.92it/s]


Dataset parquet downloaded and prepared to /Users/jiatianyu/.cache/huggingface/datasets/parquet/wikipedia-1ac450b58ffe6672/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.
Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 1250308
})


In [9]:
# 拼接数据集，同时将数据集保存到本地文件中

# 仅保留 'text' 列
wiki = wiki.remove_columns([col for col in wiki.column_names if col != "text"])
dataset = concatenate_datasets([bookcorpus, wiki])
print(dataset)

# 拆分并将数据集保存到本地文件中
d = dataset.train_test_split(test_size=0.1)

def dataset_to_text(dataset, output_filename="data.txt"):
    with open(output_filename, "w") as f:
        for t in dataset["text"]:
            print(t, file=f)
# save the training set to train.txt
dataset_to_text(d["train"], "data/train.txt")
# save the testing set to test.txt
dataset_to_text(d["test"], "data/test.txt")

Dataset({
    features: ['text'],
    num_rows: 3250308
})


> 训练词元分析器(Tokenizer)

BERT采用了`WordPiece`分词，根据训练语料中的词频决定是否将一个完整的词切分为多个词元。因此，需要首先训练词元分析器(Tokenizer)。可以使用transformers库中的`BertWordPieceTokenizer`类来完成任务